In [1]:
import os
import pandas as pd
from scipy import stats
import errno

In [2]:
data_dir = '/Users/gareth/Documents/COVID19/data_deidentified/v0.3.0_2020-07-28/'

In [3]:
daily_df = pd.read_csv(os.path.join(data_dir, 'daily.csv'))
daily = daily_df[:]

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (110,112,118,127,142,146,148,149,150) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
patients_df = pd.read_csv(os.path.join(data_dir, 'patients.csv'))
patients = patients_df[:]

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (162,221,242) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Admission data

In [5]:
patients_cols = [
    'weight', 'comorbidity_obesity', 'comorbidity_asthma',
    'comorbidity_chronic_cardiac_disease', 'comorbidity_chronic_kidney_disease',
    'age_five_year_intervals', 'final_outcome_death', 'sex', 'pin', 'date_discharge',
    'date_outcome'
]

In [6]:
# filter out unknown outcome patients
patients = patients[patients['final_outcome_unknown'] != 1]

In [7]:
patients = patients[patients_cols].dropna()

In [8]:
patients.shape

(402, 11)

### Daily data

In [9]:
daily_cols = [
    'pin', 'date_daily', 'day', 
    'serum_creatinine', 'haemoglobin', 'platelet_count', 'glucose', 
    'SaO2', 'systolic_pressure', 'diastolic_pressure', 'blood_urea_nitrogen',
     'pH', #'PaCO2', 'PaO2', 'PaO2/FiO2', 'eotd_respiratory_rate', 
]

In [10]:
daily = daily[daily.pin.isin(patients.pin)][daily_cols].dropna()

In [11]:
# this removes too much data
drop_pins = []

for pin in daily['pin'].unique():
    if (max(daily[daily.pin == pin]['date_daily']) == patients[patients.pin == pin]['date_outcome'].values[0] or
            max(daily[daily.pin == pin]['date_daily']) == patients[patients.pin == pin]['date_discharge'].values[0]):
        continue
#     drop_pins.append(pin)

In [12]:
drop_daily = daily[~daily.pin.isin(drop_pins)]

In [13]:
drop_daily = drop_daily[drop_daily.day > 0]

In [14]:
drop_daily = drop_daily.merge(patients, on='pin')

Map to the following column names 

"Creatinine", "HCO3", "HR", "Haemoglobin", "Platelet_count",
"Potassium", "RR", "Sodium", "AGE", "GENDER",
'Chronic Kidney Disease', 'Obesity', 'Heart disease', 'Asthma', 
'Weight_kg', 'SaO2', 'HADM_ID', 'DEATH'

In [15]:
drop_daily.columns

Index(['pin', 'date_daily', 'day', 'serum_creatinine', 'haemoglobin',
       'platelet_count', 'glucose', 'SaO2', 'systolic_pressure',
       'diastolic_pressure', 'blood_urea_nitrogen', 'pH', 'weight',
       'comorbidity_obesity', 'comorbidity_asthma',
       'comorbidity_chronic_cardiac_disease',
       'comorbidity_chronic_kidney_disease', 'age_five_year_intervals',
       'final_outcome_death', 'sex', 'date_discharge', 'date_outcome'],
      dtype='object')

In [16]:
col_map = {
    'pin':'HADM_ID',
    'serum_creatinine': 'Creatinine',
    'haemoglobin': 'Haemoglobin',
    'platelet_count': 'Platelet_count', 
    'final_outcome_death': 'DEATH',
    'sex': 'GENDER',
    'weight': 'Weight_kg',
    'comorbidity_chronic_cardiac_disease': 'Heart disease',
    'comorbidity_chronic_kidney_disease': 'Chronic Kidney Disease',
    'age_five_year_intervals': 'AGE',
    'comorbidity_asthma': 'Asthma',
    'comorbidity_obesity': 'Obesity',
    'day': 'Day'
}

In [17]:
drop_daily.columns = list(map(lambda x: col_map[x] if x in col_map else x, list(drop_daily.columns)))

In [18]:
drop_daily.shape

(2438, 22)

In [19]:
drop_daily.HADM_ID.unique().shape

(303,)

In [20]:
drop_daily.to_csv('covid.csv')